In [29]:
from torch import Tensor 


import torch 
import gpytorch 
import geometric_kernels.torch 
from math import comb 
from spherical_harmonics import SphericalHarmonics
from geometric_kernels.spaces import Hypersphere
from gpytorch.variational import CholeskyVariationalDistribution
from gpytorch.distributions import MultivariateNormal
from linear_operator.operators import DiagLinearOperator, LinearOperator
from mdgp.kernels import GeometricMaternKernel
from tqdm.autonotebook import tqdm 
from gpytorch.metrics import negative_log_predictive_density, mean_squared_error


torch.set_default_dtype(torch.float64)

/tmp/ipykernel_105226/3809226695.py:14: TqdmWarning:

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html



In [2]:
def num_harmonics_single(ell: int, d: int) -> int:
    r"""
    Number of spherical harmonics of degree ell on S^{d - 1}.
    """
    if ell == 0:
        return 1
    if d == 3:
        return 2 * ell + 1
    else:
        return (2 * ell + d - 2) * comb(ell + d - 3, ell - 1) // ell


def num_harmonics(ell: Tensor | int, d: int) -> Tensor:
    """
    Vectorized version of num_harmonics_single
    """
    if isinstance(ell, int):
        return num_harmonics_single(ell, d)
    return ell.apply_(lambda e: num_harmonics_single(ell=e, d=d))


def total_num_harmonics(max_ell: int, d: int) -> int:
    """
    Total number of spherical harmonics on S^{d-1} with degree <= max_ell
    """
    return int(sum(num_harmonics(ell=torch.arange(max_ell + 1), d=d)))


def eigenvalue_laplacian(ell: Tensor, d: int) -> Tensor:
    """
    Eigenvalue of the Laplace-Beltrami operator for a spherical harmonic of degree ell on S_{d-1}
    ell: [...]
    d: []
    return: [...]
    """
    return ell * (ell + d - 2)


def unnormalized_matern_spectral_density(n: Tensor, d: int, kappa: Tensor | float, nu: Tensor | float) -> Tensor | float: 
    """
    compute (unnormalized) spectral density of the matern kernel on S_{d-1}
    n: [N]
    d: []
    kappa: [O, 1, 1]
    nu: [O, 1, 1]
    return: [O, 1, N]
    """
    # Squared exponential kernel 
    if torch.all(nu.isinf()):
        exponent = -kappa ** 2 / 2 * eigenvalue_laplacian(ell=n, d=d) # [O, N, 1]
        return torch.exp(exponent)
    # Matern kernel
    else:
        base = (
            2.0 * nu / kappa**2 + # [O, 1, 1]
            eigenvalue_laplacian(ell=n, d=d).unsqueeze(-1) # [N, 1]
        ) # [O, N, 1]
        exponent = -nu - (d - 1) / 2.0 # [O, 1, 1]
        return base ** exponent # [O, N, 1]


def matern_spectral_density_normalizer(d: int, max_ell: int, kappa: Tensor | float, nu: Tensor | float) -> Tensor:
    """
    Normalizing constant for the spectral density of the Matern kernel on S^{d-1}. 
    Depends on kappa and nu. Also depends on max_ell, as truncation of the infinite 
    sum from Karhunen-Loeve decomposition. 
    """
    n = torch.arange(max_ell + 1)
    spectral_values = unnormalized_matern_spectral_density(n=n, d=d, kappa=kappa, nu=nu) # [O, max_ell + 1, 1]
    num_harmonics_per_level = num_harmonics(torch.arange(max_ell + 1), d=d).type(spectral_values.dtype) # [max_ell + 1]
    normalizer = spectral_values.mT @ num_harmonics_per_level # [O, 1, max_ell + 1] @ [max_ell + 1] -> [O, 1]
    return normalizer.unsqueeze(-2) # [O, 1, 1]


def matern_spectral_density(n: Tensor, d: int, kappa: Tensor, nu: Tensor, max_ell: int, sigma: float = 1.0) -> Tensor:
    """
    Spectral density of the Matern kernel on S^{d-1}
    """
    return (
        unnormalized_matern_spectral_density(n=n, d=d, kappa=kappa, nu=nu) / # [O, N, 1]
        matern_spectral_density_normalizer(d=d, max_ell=max_ell, kappa=kappa, nu=nu) * # [O, 1, 1]
        (sigma ** 2)[..., *(None,) * (kappa.ndim - 1)] # [O, 1, 1]
    ) # [O, N, 1] / [O, 1, 1] * [O, 1, 1] -> [O, N, 1]


def matern_ahat(ell: Tensor, d: int, max_ell: int, kappa: Tensor | float, nu: Tensor | float, 
                m: int | None = None, sigma: Tensor | float = 1.0) -> float:
    """
    :math: `\hat{a} = \rho(\ell)` where :math: `\rho` is the spectral density on S^{d-1}
    """
    return matern_spectral_density(n=ell, d=d, kappa=kappa, nu=nu, max_ell=max_ell, sigma=sigma) # [O, N, 1]


def matern_repeated_ahat(max_ell: int, d: int, kappa: Tensor | float, nu: Tensor | float, sigma: Tensor | float = 1.0) -> Tensor:
    """
    Returns a tensor of repeated ahat values for each ell. 
    """
    ells = torch.arange(max_ell + 1) # [max_ell + 1]
    ahat = matern_ahat(ell=ells, d=d, max_ell=max_ell, kappa=kappa, nu=nu, sigma=sigma) # [O, max_ell + 1, 1]
    repeats = num_harmonics(ell=ells, d=d) # [max_ell + 1]
    return torch.repeat_interleave(ahat, repeats=repeats, dim=-2) # [O, num_harmonics, 1]


def matern_Kuu(max_ell: int, d: int, kappa: float, nu: float, sigma: float = 1.0) -> Tensor: 
    """
    Returns the covariance matrix, which is a diagonal matrix with entries 
    equal to inv_ahat of the corresponding ell. 
    """
    return torch.diag(1 / matern_repeated_ahat(max_ell, d, kappa, nu, sigma=sigma).squeeze(-1)) # [O, num_harmonics, num_harmonics]


def spherical_harmonics(x: Tensor, max_ell: int, d: int) -> Tensor: 
    # Make sure that x is at least 2d and flatten it
    x = torch.atleast_2d(x)
    batch_shape, n = x.shape[:-2], x.shape[-2]
    x = x.flatten(0, -2)

    # Get spherical harmonics callable
    f = SphericalHarmonics(dimension=d, degrees=max_ell + 1) # [... * O, N, num_harmonics]

    # Evaluate x and reintroduce batch dimensions
    return f(x).reshape(*batch_shape, n, total_num_harmonics(max_ell, d)) # [..., O, N, num_harmonics]


def matern_Kux(x: Tensor, max_ell: int, d: int) -> Tensor: 
    return spherical_harmonics(x, max_ell=max_ell, d=d).mT # [..., O, num_harmonics, N]


def num_spherical_harmonics_to_degree(num_spherical_harmonics: int, dimension: int) -> tuple[int, int]:
    """
    Returns the minimum degree for which there are at least
    `num_eigenfunctions` in the collection.
    """
    n, degree = 0, 0  # n: number of harmonics, d: degree (or level)
    while n < num_spherical_harmonics:
        n += num_harmonics(d=dimension, ell=degree)
        degree += 1

    if n > num_spherical_harmonics:
        print(
            "The number of spherical harmonics requested does not lead to complete "
            "levels of spherical harmonics. We have thus increased the number to "
            f"{n}, which includes all spherical harmonics up to degree {degree} (incl.)"
        )
    return degree - 1, n


from gpytorch.kernels import Kernel, ScaleKernel
from mdgp.kernels.matern_kernel import _GeometricMaternKernel


def matern_LT_Phi(x: Tensor, max_ell: int, d: int, kappa: float, nu: float, sigma: float = 1.0) -> Tensor: 
    Kux = matern_Kux(x, max_ell=max_ell, d=d) # [..., O, num_harmonics, N]
    ahat_sqrt = matern_repeated_ahat(max_ell=max_ell, d=d, kappa=kappa, nu=nu, sigma=sigma).sqrt() # [O, num_harmonics, 1]
    return Kux * ahat_sqrt # [..., O, num_harmonics, N]


def matern_LT_Phi_from_kernel(x: Tensor, covar_module: Kernel, num_levels: int) -> Tensor: 
    # Extract kernel parameters  
    if isinstance(covar_module, ScaleKernel):
        sigma = covar_module.outputscale.sqrt()
        base_kernel = covar_module.base_kernel
    else:
        sigma = torch.tensor(1.0, dtype=x.dtype, device=x.device)
        base_kernel = covar_module
    kappa = base_kernel.lengthscale
    nu = base_kernel.nu 

    # Extract constants 
    d = base_kernel.space.dimension + 1
    max_ell = num_levels
    
    return matern_LT_Phi(x, max_ell=max_ell, d=d, kappa=kappa, nu=nu, sigma=sigma)

In [21]:
class gpytorchSGP(gpytorch.models.ApproximateGP):
    def __init__(self, max_ell, d, max_ell_prior, epsilon_sigma=1.0, kappa=1.0, nu=2.5, sigma=1.0, batch_shape=torch.Size([]), jitter_val=1e-6, optimize_nu: bool = True):
        m = total_num_harmonics(max_ell, d)
        variational_distribution = CholeskyVariationalDistribution(num_inducing_points=m, batch_shape=batch_shape)
        variational_strategy = SGPVariationalStrategy(self, variational_distribution, num_levels=max_ell, jitter_val=jitter_val)
        super().__init__(variational_strategy=variational_strategy)

        # constants 
        self.jitter_val = jitter_val
        self.max_ell = max_ell
        self.d = d

        # modules 
        base_kernel = GeometricMaternKernel(
            space=Hypersphere(d - 1),
            lengthscale=kappa, 
            nu=nu, 
            trainable_nu=optimize_nu, 
            num_eigenfunctions=max_ell_prior + 1,
        )
        self.covar_module = gpytorch.kernels.ScaleKernel(base_kernel)
        self.mean_module = gpytorch.means.ConstantMean()
        self.likelihood = gpytorch.likelihoods.GaussianLikelihood()

        # prior hyperparams 
        self.covar_module.outputscale = sigma ** 2
        self.likelihood.noise = epsilon_sigma ** 2

    def forward(self, x) -> MultivariateNormal:
        p_sigma = self.covar_module(x)
        p_sigma = p_sigma.add_jitter(self.jitter_val)
        p_mu = self.mean_module(x)
        return MultivariateNormal(p_mu, p_sigma)
    
    def K(self, x):
        p = self.forward(x)
        return p.covariance_matrix


class SGPVariationalStrategy(torch.nn.Module):
    def __init__(self, model: gpytorchSGP, variational_distribution: CholeskyVariationalDistribution, num_levels, jitter_val=1e-6):
        super().__init__()
        object.__setattr__(self, "_model", model)
        self._variational_distribution = variational_distribution
        self.jitter_val = jitter_val

        # Extract references to kernel parameters 
        self.num_levels = num_levels

    @property
    def model(self) -> gpytorchSGP:
        return self._model
    
    @property
    def variational_distribution(self) -> MultivariateNormal:
        return self._variational_distribution()
    
    @property
    def prior_distribution(self) -> MultivariateNormal:
        zeros = torch.zeros(
            self._variational_distribution.shape(),
            dtype=self._variational_distribution.dtype,
            device=self._variational_distribution.device,
        )
        ones = torch.ones_like(zeros)
        res = MultivariateNormal(zeros, DiagLinearOperator(ones))
        return res

    def forward(self, x) -> MultivariateNormal:
        # prior at x 
        px = self.model.forward(x)
        mu_x, Kxx = px.mean, px.lazy_covariance_matrix

        # whitened prior at u
        Linv_pu = self.prior_distribution
        Linv_mu, Linv_Kuu_LTinv = Linv_pu.mean, Linv_pu.lazy_covariance_matrix

        # whitened variational posterior at u
        Linv_qu = self.variational_distribution
        Linv_m, Linv_S_LTinv = Linv_qu.mean, Linv_qu.lazy_covariance_matrix

        # unwhitening + projection and vice-versa
        LT_Phiux = matern_LT_Phi_from_kernel(x, self.model.covar_module, num_levels=self.num_levels)
        Phixu_L = LT_Phiux.mT

        # posterior at x 
        qx_sigma = Kxx + Phixu_L @ (Linv_S_LTinv - Linv_Kuu_LTinv) @ LT_Phiux
        qx_sigma = qx_sigma.add_jitter(self.jitter_val)
        qx_mu = mu_x + Phixu_L @ (Linv_m - Linv_mu)
        return MultivariateNormal(qx_mu, qx_sigma)
    
    def kl_divergence(self):
        with gpytorch.settings.max_preconditioner_size(0):
            kl_divergence = torch.distributions.kl.kl_divergence(self.variational_distribution, self.prior_distribution)
        return kl_divergence
    
    def __call__(self, x, prior: bool = False, **kwargs) -> MultivariateNormal:
        if prior:
            return self.model.forward(x)
        return super().__call__(x, **kwargs)

In [22]:
from linear_operator.operators import DiagLinearOperator
from gpytorch.distributions import MultivariateNormal


class SphereProjector(torch.nn.Module):
    def __init__(self, b: float = 2.0):
        super().__init__()
        self.b = torch.nn.Parameter(torch.tensor(b))
        self.norm = None 

    def forward(self, x: Tensor, y: Tensor | None = None) -> tuple[Tensor, Tensor] | Tensor:
        b = self.b.expand(*x.shape[:-1], 1)
        x_cat_b = torch.cat([x, b], dim=-1)
        self.norm = x_cat_b.norm(dim=-1, keepdim=True)
        if y is None:
            return x_cat_b / self.norm
        else:
            return x_cat_b / self.norm, y / self.norm.squeeze(-1)
    
    def inverse(self, mvn: MultivariateNormal) -> MultivariateNormal:
        L = DiagLinearOperator(self.norm.squeeze(-1))
        mean = mvn.mean @ L
        cov = L @ mvn.lazy_covariance_matrix @ L
        return MultivariateNormal(mean=mean, covariance_matrix=cov)

In [23]:
def test_psd(S, tol=1e-8):
    if isinstance(S, LinearOperator):
        S = S.to_dense()
    assert torch.allclose(S, S.mT), "K should be symmetric."

    eigs = torch.linalg.eigvalsh(S)
    assert (eigs > -tol).all(), "K should be positive definite."

# UCI data

In [55]:
import os
import pandas as pd 
import torch 
from torch import Tensor
from torch.utils.data import TensorDataset, DataLoader, Dataset


class UCIDataset:

    UCI_BASE_URL = 'https://archive.ics.uci.edu/ml/machine-learning-databases/'

    def __init__(self, name: str, url: str, path: str = '../../data/uci/', normalize: bool = True, seed: int | None = None): 
        self.name = name 
        self.url = url
        self.path = path 
        self.csv_path = os.path.join(self.path, self.name + '.csv')

        # Set generator if seed is provided.
        self.generator = torch.Generator()
        if seed is not None: 
            self.generator.manual_seed(seed)

        # Load, shuffle, split, and normalize data. TODO except for load, these don't need to be object methods. 
        # We keep the standard deviation of the test set for log-likelihood evaluation.
        x, y = self.load_data()
        x, y = self.shuffle(x, y, generator=self.generator)     
        self.train_x, self.train_y, self.test_x, self.test_y = self.split(x, y)
        self.test_y_std = self.test_y.std(dim=0, keepdim=True)
        self.train_x, self.train_y, self.test_x, self.test_y = map(
            self.normalize, (self.train_x, self.train_y, self.test_x, self.test_y))

    @property
    def dimension(self) -> int:
        return self.train_x.shape[-1]

    @property 
    def train_dataset(self) -> Dataset:
        return TensorDataset(self.train_x, self.train_y)
    
    @property
    def test_dataset(self) -> Dataset:
        return TensorDataset(self.test_x, self.test_y)

    def read_data(self) -> tuple[Tensor, Tensor]:
        xy = torch.from_numpy(pd.read_csv(self.csv_path).values)
        return xy[:, :-1], xy[:, -1]

    def download_data(self) -> None:
        NotImplementedError

    def load_data(self, overwrite: bool = False) -> tuple[Tensor, Tensor]:
        if overwrite or not os.path.isfile(self.csv_path):
            self.download_data()
        return self.read_data()

    def normalize(self, x: Tensor) -> Tensor:
        return (x - x.mean(dim=0)) / x.std(dim=0, keepdim=True)
    
    def shuffle(self, x: Tensor, y: Tensor, generator: torch.Generator) -> tuple[Tensor, Tensor]:
        perm_idx = torch.randperm(x.size(0), generator=generator)
        return x[perm_idx], y[perm_idx]
    
    def split(self, x: Tensor, y: Tensor, test_size: float = 0.1) -> tuple[Tensor, Tensor, Tensor, Tensor]: 
        """
        Split the dataset into train and test sets.
        """
        split_idx = int(test_size * x.size(0))
        return x[split_idx:], y[split_idx:], x[:split_idx], y[:split_idx]


class Kin8mn(UCIDataset):

    DEFAULT_URL = 'https://raw.githubusercontent.com/liusiyan/UQnet/master/datasets/UCI_datasets/kin8nm/dataset_2175_kin8nm.csv'

    def __init__(self, path: str = '../../data/uci/', normalize: bool = True, seed: int | None = None, url: str | None = None):
        url = url or Kin8mn.DEFAULT_URL
        super().__init__(name='kin8nm', path=path, normalize=normalize, url=url, seed=seed)

    def download_data(self) -> None:
        df = pd.read_csv(self.url)
        os.makedirs(self.path, exist_ok=True)
        df.to_csv(self.csv_path, index=False)


import csv 
from io import BytesIO
from urllib.request import urlopen
from zipfile import ZipFile


uci_base = 'https://archive.ics.uci.edu/ml/machine-learning-databases/'


class Power(UCIDataset):

    DEFAULT_URL = uci_base + "00294/CCPP.zip"

    def __init__(self, path: str = '../../data/uci/', normalize: bool = True, seed: int | None = None, url: str | None = None):
        url = url or Power.DEFAULT_URL
        super().__init__(name='power', path=path, normalize=normalize, url=url, seed=seed)

    def download_data(self):
        with urlopen(self.url) as zipresp:
            with ZipFile(BytesIO(zipresp.read())) as zfile:
                zfile.extractall('/tmp/')

        df = pd.read_excel('/tmp/CCPP//Folds5x2_pp.xlsx')
        os.makedirs(self.path, exist_ok=True)
        df.to_csv(self.csv_path, index=False)


class Concrete(UCIDataset):

    DEFAULT_URL = uci_base + 'concrete/compressive/Concrete_Data.xls'

    def __init__(self, path: str = '../../data/uci/', normalize: bool = True, seed: int | None = None, url: str | None = None):
        url = url or Concrete.DEFAULT_URL
        super().__init__(name='concrete', path=path, normalize=normalize, url=url, seed=seed)

    def download_data(self):
        df = pd.read_excel(self.url)
        os.makedirs(self.path, exist_ok=True)
        df.to_csv(self.csv_path, index=False)

### Fixed parameters as in the spherical harmonics paper

In [64]:
# Datasets 
kin8mn = Kin8mn()
power = Power()
concrete = Concrete()

# Variational parameters
dimension_to_num_harmonics_variational = {
    4: 336,
    6: 294,
    8: 210, 
}

# Prior parameters 
"""
NOTE 
- It is difficult to say what number of spherical harmonics was used for the prior.
  I set it to be the same as the number of inducing variables.
- It is also difficult to say what lengthscale initialisation was used in the paper. 
  I set it to 1.0 for now, although a lower number, e.g. 0.001, would likely be better for higher dimensions.
"""
dimension_to_num_harmonics_prior = {
    4: 336,
    6: 294,
    8: 210, 
}
nu = 1.5
optimize_nu = False
kappa = 1.0

# Other model parameters
batch_shape = torch.Size([])

# Training parameters 
batch_size = 256 
LR = 0.01
NUM_EPOCHS = 20

In [65]:
from mdgp.utils.spherical_harmonic_features import num_spherical_harmonics_to_degree


def get_model_and_projector(dataset: UCIDataset):
    sphere_dimension = dataset.dimension + 1

    # number of levels for variational inference 
    num_spherical_harmonics = dimension_to_num_harmonics_variational[dataset.dimension]
    max_ell, _ = num_spherical_harmonics_to_degree(num_spherical_harmonics, sphere_dimension)

    # number of levels for prior
    num_spherical_harmonics_prior = dimension_to_num_harmonics_prior[dataset.dimension]
    max_ell_prior, _ = num_spherical_harmonics_to_degree(num_spherical_harmonics_prior, sphere_dimension)

    model = gpytorchSGP(max_ell=max_ell, d=sphere_dimension, max_ell_prior=max_ell_prior, kappa=kappa, nu=nu, optimize_nu=optimize_nu, batch_shape=batch_shape)
    projector = SphereProjector()
    return model, projector


model, projector = get_model_and_projector(kin8mn)

In [79]:
def train_step(x, y, model, projector, optimizer, mll) -> float:
    optimizer.zero_grad(set_to_none=True)
    x, y = projector(x, y)
    output = model(x)
    loss = mll(output, y)
    loss.backward()
    optimizer.step()
    return loss.item()


def train(dataset, model, projector, num_epochs=NUM_EPOCHS, lr=LR) -> list[float]: 
    # optimizer and criterion
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, maximize=True)
    mll = gpytorch.mlls.VariationalELBO(model.likelihood, model, dataset.train_y.size(0))

    # data
    train_loader = DataLoader(dataset.train_dataset, batch_size=batch_size, shuffle=True)

    # Training loop
    losses = []
    pbar = tqdm(range(num_epochs))
    for epoch in pbar:
        epoch_loss = 0
        for x_batch, y_batch in train_loader:
            loss = train_step(x=x_batch, y=y_batch, model=model, projector=projector, optimizer=optimizer, mll=mll)
            epoch_loss += loss
        losses.append(epoch_loss)
        pbar.set_postfix({'ELBO': losses[-1]})

    return losses 


def evaluate(dataset, model, projector):
    with torch.no_grad():
        test_x, test_y = projector(dataset.test_x), dataset.test_y
        out = model.likelihood(model(test_x))
        out = projector.inverse(out)
        nlpd = negative_log_predictive_density(out, test_y)
        mse = mean_squared_error(out, test_y)
        metrics = {
            'nlpd': nlpd.item(), 
            'mse': mse.item(),
        }
        print(f"NLPD: {metrics['nlpd']}, MSE: {metrics['mse']}")
    return metrics 

In [86]:
def reproduce_results(dataset, num_runs: int = 5, num_epochs=NUM_EPOCHS, lr=LR):
    print(f"Reproducing results for {dataset.name}".center(80, '-') + '\n')

    metrics = []
    for run in range(num_runs):
        print(f"Run {run + 1}".center(80, '-'))

        torch.random.manual_seed(run)
        model, projector = get_model_and_projector(dataset)
        train(dataset, model, projector, num_epochs=num_epochs, lr=lr)
        run_metrics = evaluate(dataset, model, projector)
        metrics.append(run_metrics)
    df = pd.DataFrame(metrics)

    print("Metrics mean".center(80, '-'))
    print(df.mean())

    print("Metrics STD".center(80, '-'))
    print(df.std())

    return df 

In [87]:
reproduce_results(kin8mn, num_epochs=20)

-------------------------Reproducing results for kin8nm-------------------------

-------------------------------------Run 1--------------------------------------


100%|██████████| 20/20 [01:46<00:00,  5.34s/it, ELBO=15.1]


NLPD: 0.6032434687912238, MSE: 0.20803978335338094
-------------------------------------Run 2--------------------------------------


100%|██████████| 20/20 [01:47<00:00,  5.39s/it, ELBO=15]  


NLPD: 0.6209709920994478, MSE: 0.21654226769310206
-------------------------------------Run 3--------------------------------------


...

100%|██████████| 20/20 [01:47<00:00,  5.39s/it, ELBO=15]  


NLPD: 0.6208321962913345, MSE: 0.2163702219899631
-------------------------------------Run 5--------------------------------------


100%|██████████| 20/20 [01:46<00:00,  5.31s/it, ELBO=15]  


NLPD: 0.6045537219725988, MSE: 0.2049825238771969
----------------------------------Metrics mean----------------------------------
nlpd    0.611172
mse     0.210723
dtype: float64
----------------------------------Metrics STD-----------------------------------
nlpd    0.008946
mse     0.005365
dtype: float64


,nlpd,mse
0,0.603243,0.208040
1,0.620971,0.216542
2,0.606258,0.207682
3,0.620832,0.216370
4,0.604554,0.204983


In [88]:
reproduce_results(power, num_epochs=20)

-------------------------Reproducing results for power--------------------------

-------------------------------------Run 1--------------------------------------


100%|██████████| 20/20 [02:20<00:00,  7.04s/it, ELBO=34]  


NLPD: 0.018200022938892634, MSE: 0.0604323361222259
-------------------------------------Run 2--------------------------------------


100%|██████████| 20/20 [02:12<00:00,  6.63s/it, ELBO=34]  


NLPD: 0.020585911675719538, MSE: 0.06111090834631231
-------------------------------------Run 3--------------------------------------


...

100%|██████████| 20/20 [02:17<00:00,  6.89s/it, ELBO=34]  


NLPD: 0.011682565872741057, MSE: 0.05958469777710783
-------------------------------------Run 5--------------------------------------


100%|██████████| 20/20 [02:14<00:00,  6.71s/it, ELBO=34.2]


NLPD: 0.018373846236252843, MSE: 0.06075983045730417
----------------------------------Metrics mean----------------------------------
nlpd    0.019651
mse     0.060782
dtype: float64
----------------------------------Metrics STD-----------------------------------
nlpd    0.006391
mse     0.000895
dtype: float64


,nlpd,mse
0,0.018200,0.060432
1,0.020586,0.061111
2,0.029411,0.062023
3,0.011683,0.059585
4,0.018374,0.060760


In [89]:
reproduce_results(concrete, num_epochs=125)

------------------------Reproducing results for concrete------------------------

-------------------------------------Run 1--------------------------------------


100%|██████████| 125/125 [01:26<00:00,  1.44it/s, ELBO=2.44] 


NLPD: 0.3623051474366128, MSE: 0.1097215590579564
-------------------------------------Run 2--------------------------------------


100%|██████████| 125/125 [01:32<00:00,  1.35it/s, ELBO=2.46] 


NLPD: 0.3629351313556932, MSE: 0.10951533423101913
-------------------------------------Run 3--------------------------------------


...

100%|██████████| 125/125 [01:28<00:00,  1.41it/s, ELBO=2.46] 


NLPD: 0.37304157369488944, MSE: 0.1139838157709856
-------------------------------------Run 5--------------------------------------


100%|██████████| 125/125 [01:28<00:00,  1.41it/s, ELBO=2.41] 


NLPD: 0.37610820039944604, MSE: 0.11769848257163341
----------------------------------Metrics mean----------------------------------
nlpd    0.370684
mse     0.113703
dtype: float64
----------------------------------Metrics STD-----------------------------------
nlpd    0.007663
mse     0.004018
dtype: float64


,nlpd,mse
0,0.362305,0.109722
1,0.362935,0.109515
2,0.379029,0.117594
3,0.373042,0.113984
4,0.376108,0.117698
